<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [40]:
import requests
import json
from datetime import datetime, date, time
import numpy as np

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [4]:
print(response.status_code)

200


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [9]:
parameters = {'lat':-33.87, 'lon':151.21}
print(parameters['lat'])

-33.87


The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [14]:
response = requests.get("http://api.open-notify.org/iss-pass.json",parameters)

Print the response header:

In [21]:
#print(response.json)
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Fri, 09 Oct 2020 09:16:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [19]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1602234716, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 163, \n      "risetime": 1602238505\n    }, \n    {\n      "duration": 646, \n      "risetime": 1602244025\n    }, \n    {\n      "duration": 606, \n      "risetime": 1602249860\n    }, \n    {\n      "duration": 411, \n      "risetime": 1602255835\n    }, \n    {\n      "duration": 371, \n      "risetime": 1602261769\n    }\n  ]\n}\n'


Note that this is a Python byte string:

Print just the "content-type" value from the header:

In [25]:
print(response.headers['Content-Type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [33]:
returned_value = response.json()
print(returned_value)
print(returned_value['message'])

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1602234716, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 163, 'risetime': 1602238505}, {'duration': 646, 'risetime': 1602244025}, {'duration': 606, 'risetime': 1602249860}, {'duration': 411, 'risetime': 1602255835}, {'duration': 371, 'risetime': 1602261769}]}
success


What kind of object did this give us?

In [34]:
print(type(returned_value))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [36]:
response_values = returned_value['response']
print(response_values)

[{'duration': 163, 'risetime': 1602238505}, {'duration': 646, 'risetime': 1602244025}, {'duration': 606, 'risetime': 1602249860}, {'duration': 411, 'risetime': 1602255835}, {'duration': 371, 'risetime': 1602261769}]


Now extract the `risetime` strings into an array called `srisetimes`:

In [44]:
srisetimes = np.array([item['risetime'] for item in response_values])

print(type(srisetimes))
print(srisetimes)

<class 'numpy.ndarray'>
[1602238505 1602244025 1602249860 1602255835 1602261769]


```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [50]:
risetimes = np.array([datetime.fromtimestamp(time) for time in srisetimes])

print(risetimes)

[datetime.datetime(2020, 10, 9, 21, 15, 5)
 datetime.datetime(2020, 10, 9, 22, 47, 5)
 datetime.datetime(2020, 10, 10, 0, 24, 20)
 datetime.datetime(2020, 10, 10, 2, 3, 55)
 datetime.datetime(2020, 10, 10, 3, 42, 49)]


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [51]:
print([time.strftime('%d/%m/%y %I:%M') for time in risetimes])

['09/10/20 09:15', '09/10/20 10:47', '10/10/20 12:24', '10/10/20 02:03', '10/10/20 03:42']


Here is an endpoint that tells us who is onboard:

In [52]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [56]:
returned_json = response.json()
print(returned_json)
print('Numnber of people onboard = {}'.format(returned_json['number']))

people_info = returned_json['people']
people_name = [item['name'] for item in people_info]
print(people_name)

{'number': 3, 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'}, {'craft': 'ISS', 'name': 'Anatoly Ivanishin'}, {'craft': 'ISS', 'name': 'Ivan Vagner'}], 'message': 'success'}
Numnber of people onboard = 3
['Chris Cassidy', 'Anatoly Ivanishin', 'Ivan Vagner']


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [58]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
    return([response.status_code!=200,response.status_code])
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [60]:
response = requests.get("http://api.open-notify.org/astros.json")
print(handleResponse(response)[0])
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

False


True

>

>

>



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



